In [32]:
import tensorflow as tf
from tensorflow.keras import datasets, layers
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np


# Check version of TensorFlow (exam requires a certain version)
# See for version: https://www.tensorflow.org/extras/cert/Setting_Up_TF_Developer_Certificate_Exam.pdf
print(tf.__version__)

2.8.2


In [33]:
# Get data
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()

print(type(train_images), type(train_labels))
print(train_images.shape, train_labels.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)


In [34]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [43]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

In [35]:
class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

Class names: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [36]:
for image, label in train_dataset.take(1):
  print(type(image), type(label))
  print(image.shape, label.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>
(28, 28, 1) ()


In [37]:
# Normalize images (get values between 0 & 1)
train_images, test_images = train_images / 255.0, test_images / 255.0

In [38]:
print(train_images.shape, tf.expand_dims(train_images, axis=-1).shape)

(60000, 28, 28) (60000, 28, 28, 1)


In [39]:
# Check shape of input data
print(train_images.shape)
print(train_labels.shape)

#Input shape
X_train = np.expand_dims(train_images, axis=-1)
y_train = np.expand_dims(train_labels, axis=-1)
X_test = np.expand_dims(test_images, axis=-1)
y_test = np.expand_dims(test_labels, axis=-1)


# Check shape of input data
print(X_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)
(60000, 28, 28, 1)
(60000, 1)


In [40]:
# Build model
model = tf.keras.Sequential([
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPool2D(),
    layers.Conv2D(32, 3, activation="relu"),
    layers.Flatten(), # flatten outputs of final Conv layer to be suited for final Dense layer
    layers.Dense(10, activation="softmax")
])

# Compile model
model.compile(loss="sparse_categorical_crossentropy", # if labels aren't one-hot use sparse (if labels are one-hot, drop sparse)
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit model
print("Training model...")
history = model.fit(x=X_train,
          y=y_train,
          epochs=10,
          validation_data=(X_test, y_test))


Training model...
Epoch 1/10
1875/1875 [==============================] - 56s 30ms/step - loss: 0.5382 - accuracy: 0.8019 - val_loss: 0.4108 - val_accuracy: 0.8504
Epoch 2/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.3601 - accuracy: 0.8712 - val_loss: 0.3632 - val_accuracy: 0.8692
Epoch 3/10
1875/1875 [==============================] - 51s 27ms/step - loss: 0.3138 - accuracy: 0.8872 - val_loss: 0.3278 - val_accuracy: 0.8822
Epoch 4/10
1875/1875 [==============================] - 52s 28ms/step - loss: 0.2878 - accuracy: 0.8943 - val_loss: 0.3301 - val_accuracy: 0.8784
Epoch 5/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.2657 - accuracy: 0.9031 - val_loss: 0.3193 - val_accuracy: 0.8812
Epoch 6/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.2513 - accuracy: 0.9087 - val_loss: 0.2856 - val_accuracy: 0.8988
Epoch 7/10
1875/1875 [==============================] - 53s 28ms/step - loss: 0.2393 - accuracy: 0.9125 - 

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 32)          9248      
                                                                 
 flatten_1 (Flatten)         (None, 288)              

In [42]:
# Evaluate model
print("Evaluating model...")
model.evaluate(test_images, test_labels)

Evaluating model...
313/313 [==============================] - 3s 9ms/step - loss: 0.2924 - accuracy: 0.8947


[0.29236021637916565, 0.8946999907493591]

In [45]:
# Evaluate model
print("Evaluating model...")
test_dataset = test_dataset.cache().batch(32)

model.evaluate(test_dataset)

Evaluating model...
313/313 [==============================] - 6s 17ms/step - loss: 0.2924 - accuracy: 0.8947


[0.2923601269721985, 0.8946999907493591]

In [46]:
# Plot the validation and training data separately
def plot_loss_curves(history, metric="mae"):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history[metric]
  val_accuracy = history.history[f'val_{metric}']

  epochs = range(len(history.history['loss']))

  plt.figure(figsize=(15, 5))
  # Plot loss
  plt.subplot(1, 2, 1)
  plt.title('Loss')
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.subplot(1, 2, 2)
  plt.title(metric)
  plt.plot(epochs, accuracy, label=f'training_{metric}')
  plt.plot(epochs, val_accuracy, label=f'val_{metric}')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(history, metric="accuracy")

In [ ]:
# Save model to current working directory
model.save("test_image_model.h5")